# Movie Recommender

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Import the data

### Find one user or aggregate many users  
    1) find the most similar user or aggregate user
    2) identify that user's highly rated movies

### Metrics to judge similarity
        - based on aggregrate rating of a movie
        - based on what the rating
        - based on what genres they watched the most of
        - based on how frequently they rate movies
        - something with tags on the movie? Sentiment analysis?
        - scrape data from imdb for a critic's review
        - timestamps?

### Misc  
    - User liked a movie = T/F if their rating > aggregate movie rating
    - 

In [1]:
# set up sparkcontext
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Movie Recommender")
sc = SparkContext(conf = conf) 

In [2]:
# create ratings RDDs
ratingsRDD = sc.textFile("Data/ratings.csv")
ratingsRDD = ratingsRDD.map(lambda x: x.split(','))

In [3]:
# remove header
from itertools import islice
ratingsRDD = ratingsRDD.mapPartitionsWithIndex(lambda idx, it: islice(it, 1, None) if idx == 0 else it)

In [4]:
# convert datatypes in RDD
ratingsRDD = ratingsRDD.map(lambda x: [int(x[0]), int(x[1]), float(x[2]), int(x[3])])

In [5]:
# create key-value pairs of movieid and user rating
ratingsbymovieid = ratingsRDD.map(lambda x: (x[1], x[2]))

In [6]:
ratingsbymovieid.take(5)

[(31, 2.5), (1029, 3.0), (1061, 3.0), (1129, 2.0), (1172, 4.0)]

In [7]:
# create a tuple of (movieid, (sumofratings, numberof ratings))
averagemovierating = ratingsbymovieid.mapValues(lambda x: (x,1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [8]:
# reduced the previous data frame to (movieid, averagemovierating)
averagemovierating = averagemovierating.map(lambda x: (x[0], x[1][0]/x[1][1]))

In [9]:
averagemovierating.take(5)

[(1172, 4.260869565217392),
 (2150, 3.513888888888889),
 (2294, 3.2735849056603774),
 (2968, 3.5697674418604652),
 (10, 3.4508196721311477)]

In [10]:
def rec_func(data):
    pass